In [1]:
import numpy as np
import pandas as pd

In [2]:
import prepostprocessing.cleaning as cleaning
import prepostprocessing.pre_processing as preproc
from qapf import qapf
from qapf import cipw

___

# CIPWFULL normalized data

## Convert to CIPWFULL format

**I used an arbitrary version of the mineralogical data, not sure if this is the correct one** -->is a good one

In [3]:
mineralogy_original = pd.read_excel("../_CIPW/CIPW/AREA1/interpolated_data.xlsx", index_col=0)
# Only iterested in sample number and oxides
mineralogy = mineralogy_original[['SiO2', 'TiO2', 'Al2O3', 'Fe2O3','FeO', 'MnO', 'MgO', 'CaO', 'Na2O', 'K2O', 'oth']]
mineralogy.columns

Index(['SiO2', 'TiO2', 'Al2O3', 'Fe2O3', 'FeO', 'MnO', 'MgO', 'CaO', 'Na2O',
       'K2O', 'oth'],
      dtype='object')

In [4]:
mineralogy = mineralogy.dropna()

In [5]:
# Check for closed sum
assert all(np.isclose(mineralogy.sum(axis=1), 100.0))

In [6]:
# Remove 'l.i.'' and 'oth' columns and immediately normalize data
mineralogy = preproc.normalize(mineralogy.drop(["oth"], axis=1))

In [7]:
# Check for closed sum
assert all(np.isclose(mineralogy.sum(axis=1), 100.0))

In [8]:
mineralogy.head()

,SiO2,TiO2,Al2O3,Fe2O3,FeO,MnO,MgO,CaO,Na2O,K2O
2809,70.833273,0.308666,15.168440,0.657828,2.118629,0.039840,1.072359,2.123800,3.904487,3.772676
2810,70.851666,0.307878,15.165414,0.657862,2.113487,0.039764,1.068630,2.116367,3.904174,3.774759
2811,70.865827,0.307266,15.163057,0.657828,2.109589,0.039705,1.065761,2.110684,3.903923,3.776359
2812,70.752381,0.315020,15.112665,0.624228,2.239309,0.043214,1.044325,2.120467,3.938101,3.810289
2813,70.754667,0.314909,15.112089,0.623983,2.239052,0.043211,1.043777,2.119570,3.938109,3.810633


In [9]:
mineralogy

,SiO2,TiO2,Al2O3,Fe2O3,FeO,MnO,MgO,CaO,Na2O,K2O
2809,70.833273,0.308666,15.168440,0.657828,2.118629,0.039840,1.072359,2.123800,3.904487,3.772676
2810,70.851666,0.307878,15.165414,0.657862,2.113487,0.039764,1.068630,2.116367,3.904174,3.774759
2811,70.865827,0.307266,15.163057,0.657828,2.109589,0.039705,1.065761,2.110684,3.903923,3.776359
2812,70.752381,0.315020,15.112665,0.624228,2.239309,0.043214,1.044325,2.120467,3.938101,3.810289
2813,70.754667,0.314909,15.112089,0.623983,2.239052,0.043211,1.043777,2.119570,3.938109,3.810633
...,...,...,...,...,...,...,...,...,...,...
205097,72.198275,0.284841,14.589975,0.953028,1.723263,0.045902,0.627314,1.255755,4.071264,4.250382
205541,72.181093,0.286060,14.594273,0.963251,1.721209,0.045953,0.629252,1.257302,4.072479,4.249130
205542,72.183092,0.285914,14.593842,0.962047,1.721417,0.045944,0.629057,1.257166,4.072304,4.249218
205543,72.185431,0.285748,14.593285,0.960718,1.721633,0.045936,0.628801,1.256948,4.072127,4.249372


In [10]:
mineralogy1 = mineralogy.iloc[0:45000]
mineralogy2 = mineralogy.iloc[45001:90000]

In [11]:
cipw.convert_to_CIPWFULL_format(mineralogy1, "../_CIPW/CIPW/AREA1/area1_1.txt", dataset_name="Vistelius (1995)")
cipw.convert_to_CIPWFULL_format(mineralogy2, "../_CIPW/CIPW/AREA1/area1_2.txt", dataset_name="Vistelius (1995)")

_____

## Extract results from CIPWFULL results file

**I only selected quartz, plagioclase and alkali feldspar here as they are of interest to us for the QAPF classification**  
**Feel free to extract addiotnal minerals if they would be of any interest to you**

**Not sure what to do with albite in the final assignments as by the book it should be accounted for as an alkali feldspar and thus be included in the sum of plagioclase but then the resulting plagioclase contents seem quite low.   
This is something you might look into although the method used here seems to be correct according to te resulting QAPF diagram.**

--> indeed when reading the paper of Streckeisen (and other sources), albite should be accounted for as an alkali feldspar, also saw that the datapoints were generally located more to the right side (plagioclase), maybe an overestimation of plagioclase?

--> I changed the settings below so that albite belongs to the alkali feldspars, Think this figure should be more correct according to the papers, but the cotent of plagioclase is indeed quite low. 

In [12]:
df_final1 = cipw.extract_CIPW_results("../_CIPW/CIPW/AREA1/area1_1_results_excel.txt")
df_final2 = cipw.extract_CIPW_results("../_CIPW/CIPW/AREA1/area1_2_results_excel.txt")

C:\Users\Sebastian\Anaconda3\envs\main\lib\site-packages\IPython\core\interactiveshell.py:3254: DtypeWarning: Columns (0,1,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [13]:
df_final = pd.concat([df_final1, df_final2])

In [14]:
df_final

,Q,P,K
SAMPLE,,,
2809,26.205746,40.268396,25.596891
2810,26.243313,40.229100,25.608911
2811,26.272108,40.198968,25.618154
2812,25.722660,40.507814,25.847571
2813,25.726379,40.503427,25.849590
...,...,...,...
205097,27.714497,37.232150,28.560789
205541,27.695215,37.249129,28.554432
205542,27.697744,37.247114,28.554812


## QAPF classification

In [15]:
df_final["QAPF"] = qapf.check_QAPF_region(df_final)

In [16]:
df_final["QAPF"].unique()

array(['monzo granite', 'granodiorite',
       'quartz monzodiorite\nquartz monzogabbro'], dtype=object)

In [17]:
df_final.to_excel("../_CIPW/CIPW/AREA1/qapf.xlsx", index=True)

In [22]:
df_final

,Q,P,K,QAPF
SAMPLE,,,,
2809,26.205746,40.268396,25.596891,monzo granite
2810,26.243313,40.229100,25.608911,monzo granite
2811,26.272108,40.198968,25.618154,monzo granite
2812,25.722660,40.507814,25.847571,monzo granite
2813,25.726379,40.503427,25.849590,monzo granite
...,...,...,...,...
205097,27.714497,37.232150,28.560789,monzo granite
205541,27.695215,37.249129,28.554432,monzo granite
205542,27.697744,37.247114,28.554812,monzo granite


-------

In [18]:
df_final.iloc[:, 3].value_counts().to_excel("../_CIPW/CIPW/AREA1/QAPF_counts.xlsx")

-----

In [19]:
QAPF_control = pd.read_excel("../_CIPW/CIPW/1_normalized.xlsx", index_col=0)
Area = pd.read_excel("../_INTERPOLATION/area_subdivided.xlsx", index_col=[0], usecols = [0, 1, 2, 3, 4, 29])

In [20]:
QAPF_control = pd.concat([Area, QAPF_control], axis = 1)
QAPF_control = QAPF_control.dropna()
QAPF_control

,Y_UTM,X_UTM,ZoneNumber,ZoneLetter,area,Q,P,K,QAPF
1,7.880625e+05,4.936585e+06,53.0,T,2.0,52.526539,17.979983,22.933498,monzo granite
2,-5.134927e+05,6.119173e+06,53.0,T,1.0,45.821201,19.425633,32.374005,monzo granite
3,1.544245e+06,7.110278e+06,53.0,T,3.0,50.716606,6.475010,37.375668,syeno granite
4,1.083775e+06,6.877413e+06,53.0,T,3.0,41.397182,23.234657,27.592905,monzo granite
5,1.641085e+06,7.925964e+06,53.0,T,4.0,54.609970,18.013528,19.746868,monzo granite
...,...,...,...,...,...,...,...,...,...
4654,6.202597e+05,5.148324e+06,53.0,T,2.0,0.000000,52.913423,17.411244,monzodiorite monzogabbro
4655,1.756745e+06,7.702655e+06,53.0,T,4.0,0.000000,59.865509,15.514612,monzodiorite monzogabbro
4656,6.586139e+05,5.200781e+06,53.0,T,2.0,0.000000,52.355599,12.039131,monzodiorite monzogabbro
4658,1.177291e+06,6.811066e+06,53.0,T,3.0,0.000000,71.436466,10.147088,monzodiorite monzogabbro


In [21]:
QAPF_control_area1 = QAPF_control[QAPF_control['area'] ==1]
QAPF_control_area1.to_excel("../_CIPW/CIPW/AREA1/QAPF_control_points.xlsx", index=True)